In [ ]:
import cv2
import numpy as np
import multiprocessing as mp
import time
import matplotlib.pyplot as plt

# Fungsi konvolusi biasa (vanilla, satu proses)
def konvolusi_biasa(img, kernel):
    kH, kW = kernel.shape
    pad_h = kH // 2
    pad_w = kW // 2

    img_padded = cv2.copyMakeBorder(img, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_REPLICATE)
    output = np.zeros_like(img)

    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            roi = img_padded[y:y + kH, x:x + kW]
            output[y, x] = (roi * kernel).sum()

    return output

# Fungsi konvolusi untuk satu baris (paralel)
def konvolusi_baris(args):
    img_padded, kernel, y = args
    kH, kW = kernel.shape
    pad_w = kW // 2

    row_output = np.zeros(img_padded.shape[1] - 2 * pad_w, dtype=np.float32)
    for x in range(pad_w, img_padded.shape[1] - pad_w):
        roi = img_padded[y:y + kH, x - pad_w:x + pad_w + 1]
        row_output[x - pad_w] = (roi * kernel).sum()
    return row_output

# Fungsi konvolusi paralel
def konvolusi_paralel(img, kernel):
    kH, kW = kernel.shape
    pad_h = kH // 2
    pad_w = kW // 2

    img_padded = cv2.copyMakeBorder(img, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_REPLICATE)

    # Siapkan args untuk tiap baris
    args = [(img_padded, kernel, y) for y in range(pad_h, img_padded.shape[0] - pad_h)]

    # Buat pool multiprocessing
    with mp.Pool(processes=mp.cpu_count()) as pool:
        hasil = pool.map(konvolusi_baris, args)

    # Gabungkan hasil
    output = np.stack(hasil, axis=0)
    return output

# Main
if __name__ == "__main__":
    # Baca gambar grayscale
    img = cv2.imread('DSC_0057.JPG', cv2.IMREAD_GRAYSCALE).astype(np.float32)

    # Kernel blur sederhana 3x3
    kernel = np.ones((3, 3), dtype=np.float32) / 9.0

    # Konvolusi biasa
    start = time.time()
    hasil_vanilla = konvolusi_biasa(img, kernel)
    end = time.time()
    print(f"Waktu konvolusi VANILLA: {end - start:.4f} detik")

    # Konvolusi paralel
    start = time.time()
    hasil_paralel = konvolusi_paralel(img, kernel)
    end = time.time()
    print(f"Waktu konvolusi PARALEL: {end - start:.4f} detik")

    # Simpan hasil untuk cek manual
    cv2.imwrite('hasil_vanilla.jpg', hasil_vanilla.astype(np.uint8))
    cv2.imwrite('hasil_paralel.jpg', hasil_paralel.astype(np.uint8))


    # Tampilkan hasil
    plt.figure(figsize=(12, 6))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title('Gambar Asli')
    plt.axis('off')

    # Vanilla convolution result
    plt.subplot(1, 3, 2)
    plt.imshow(hasil_vanilla, cmap='gray')
    plt.title('Hasil Konvolusi Vanilla')
    plt.axis('off')

    # Paralel convolution result
    plt.subplot(1, 3, 3)
    plt.imshow(hasil_paralel, cmap='gray')
    plt.title('Hasil Konvolusi Paralel')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


    print("Selesai! Hasil sudah disimpan.")


Waktu konvolusi VANILLA: 54.8132 detik
